In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import json
import torch
import numpy as np
import pandas as pd
import regex  # for grapheme cluster splitting
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader

In [2]:
df = pd.read_json("/kaggle/input/thesisdata1/final_train1.json")
td = pd.read_json("/kaggle/input/thesisdata1/final_testing1.json")
td

,text,label
0,আলমের গ্লোকোমা চিকিৎসার জন্য ১২ সপ্তাহের লেজার...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ..."
1,রাটাটের ম্যাকুলার ডিজেনারেশনের চিকিৎসার জন্য ৩...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,রাটাটের ম্যাকুলার ডিজেনারেশনের চিকিৎসার জন্য ৩...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"বিএফএ-এর ২৩তম রাউন্ডে, ১,৯৩২টি পরিবারকে কল করা...","[0, 0, 0, 0, 0, 0, 2, 0, 1, 1, 1, 0, 2, 0, 0, ..."
4,গত ৩ সপ্তাহে রিয়ার রক্তচাপ কিছুটা উন্নতি দেখি...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
295,এভারেস্ট জয় করেছেন ৫২০০ জন। কে২ শৃঙ্গ জয় করে...,"[1, 0, 0, 2, 2, 0, 1, 1, 0, 0, 2, 2, 0, 1, 2, ..."
296,অ্যাপালাচিয়ান ট্রেইলের দৈর্ঘ্য ৩৫০০ কিলোমিটার...,"[1, 1, 0, 2, 2, 0, 1, 1, 1, 2, 2, 0, 1, 1, 1, ..."
297,বান্দরবানে বনভূমির পরিমাণ ১২০০০ হেক্টর। রাঙামা...,"[1, 0, 0, 2, 2, 0, 1, 2, 2, 0, 1, 2, 2, 0, 1, ..."
298,পিরামিড নির্মিত হয় প্রায় ৪৫০০ বছর আগে। স্টোন...,"[1, 0, 0, 0, 2, 2, 0, 0, 1, 2, 2, 0, 0, 1, 2, ..."


In [3]:
raw_text       = df["text"].tolist()
raw_label      = df["label"].tolist()
raw_test_text  = td["text"].tolist()
raw_test_label = td["label"].tolist()

In [4]:
model_checkpoint = "FacebookAI/xlm-roberta-large-finetuned-conll03-english"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
bert_model = AutoModel.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/852 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

In [5]:
# Hyperparameters
sequence_length = 256  # Max subword length
num_labels = 4         # e.g. 0,1,2,3
etag = 4               # Padding label
ftag = 3               # Tag for sub-word tokens

In [11]:
# ✅ Tokenization & BERT Input Formatting
def bert_input(text_list, label_list):
    input_ids_list, attention_mask_list, token_type_list, tag_list = [], [], [], []

    for sentence, labels in zip(text_list, label_list):
        tokens = custom_tokenize(sentence)  # ✅ Use custom tokenizer
        new_labels = []

        tokenized_ids = []
        for word, tag in zip(tokens, labels):
            tokenized_word = tokenizer.tokenize(word)
            token_ids = tokenizer.convert_tokens_to_ids(tokenized_word)

            labels_expanded = [tag] + ([ftag] * (len(tokenized_word) - 1))  # Extend label for subwords

            tokenized_ids.extend(token_ids)
            new_labels.extend(labels_expanded)

        # Add CLS and SEP tokens
        tokenized_ids = [tokenizer.cls_token_id] + tokenized_ids + [tokenizer.sep_token_id]
        new_labels = [0] + new_labels + [0]

        # Create attention mask & token type ids
        attention_mask = [1] * len(tokenized_ids)
        token_type_ids = [0] * len(tokenized_ids)

        # Padding
        if len(tokenized_ids) < sequence_length:
            pad_length = sequence_length - len(tokenized_ids)
            tokenized_ids.extend([0] * pad_length)
            new_labels.extend([etag] * pad_length)
            attention_mask.extend([0] * pad_length)
            token_type_ids.extend([0] * pad_length)

        elif len(tokenized_ids) > sequence_length:
            tokenized_ids = tokenized_ids[:sequence_length - 1] + [tokenizer.sep_token_id]
            new_labels = new_labels[:sequence_length - 1] + [0]
            attention_mask = attention_mask[:sequence_length]
            token_type_ids = token_type_ids[:sequence_length]

        input_ids_list.append(torch.tensor(tokenized_ids))
        attention_mask_list.append(torch.tensor(attention_mask))
        token_type_list.append(torch.tensor(token_type_ids))
        tag_list.append(torch.tensor(new_labels))

    return {
        "input_ids": torch.stack(input_ids_list),
        "attention_mask": torch.stack(attention_mask_list),
        "token_type_ids": torch.stack(token_type_list)
    }, torch.stack(tag_list)

In [12]:
# ✅ Custom Dataset Class
class NERDataset(Dataset):
    def __init__(self, texts, labels):
        self.inputs, self.labels = bert_input(texts, labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.inputs["input_ids"][idx],
            "attention_mask": self.inputs["attention_mask"][idx],
            "token_type_ids": self.inputs["token_type_ids"][idx],
            "labels": self.labels[idx]
        }

In [13]:
import random
data = list(zip(raw_text, raw_label))
tdata= list(zip(raw_test_text, raw_test_label))
random.shuffle(data)
random.shuffle(tdata)

shuffled_text,  shuffled_label  = zip(*data)
shuffled_ttext, shuffled_tlabel = zip(*tdata)

split_1 = int(0.8 * len(shuffled_text))
train_dataset = NERDataset(shuffled_text[:split_1],  shuffled_label[:split_1])
valid_dataset = NERDataset(shuffled_text[split_1:], shuffled_label[split_1:])

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=4, shuffle=True)

import torch.nn as nn
from transformers import AutoModel

class BERT_LSTM_NER(nn.Module):
    def __init__(self, bert_model, num_labels):
        super(BERT_LSTM_NER, self).__init__()
        self.bert = bert_model
        
        # 1) Dynamically get the hidden size from the model config
        hidden_dim = self.bert.config.hidden_size  # 1024 for xlm-roberta-large

        # 2) Initialize LSTM with the correct input size
        self.lstm = nn.LSTM(
            input_size=hidden_dim,
            hidden_size=512,
            num_layers=2,
            bidirectional=True,
            batch_first=True,
            dropout=0.3  
        )

        # If it's bidirectional with hidden_size=512 => output dim = 1024
        self.fc = nn.Linear(512 * 2, num_labels)

    def forward(self, input_ids, attention_mask, token_type_ids):
        # with torch.no_grad():
        bert_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )

        lstm_output, _ = self.lstm(bert_output.last_hidden_state)
        # lstm_output is [batch_size, seq_len, 1024] because hidden_size=512 + bidirectional
        
        logits = self.fc(lstm_output)
        return logits


In [14]:
test_dataset = NERDataset(shuffled_ttext, shuffled_tlabel)
test_loader  = DataLoader(test_dataset, batch_size=4, shuffle=True)

In [15]:
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BERT_LSTM_NER(bert_model, num_labels).to(device)
# optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
optimizer = optim.AdamW([
    {"params": model.bert.parameters(), "lr": 5e-6},  # Slower fine-tuning for BERT
    {"params": model.lstm.parameters(), "lr": 2e-4},  # Faster training for LSTM
    {"params": model.fc.parameters(), "lr": 2e-4}     # Fully connected layer also gets higher LR
])
criterion = torch.nn.CrossEntropyLoss(ignore_index=etag)  # Ignore padding labels

In [16]:
import torch
from torchmetrics import Precision, Recall, F1Score, Accuracy

def compute_metrics(preds, labels, ignore_index=etag):
    """
    Computes precision, recall, F1-score, and accuracy.
    Ignores padding labels.
    """
    
    # Reshape to 1D
    preds = preds.view(-1)
    labels = labels.view(-1)

    # Ignore padding labels
    mask = labels != ignore_index  
    preds, labels = preds[mask], labels[mask]  # Apply mask

    p = precision(preds, labels)
    r = recall(preds, labels)
    f = f1(preds, labels)
    a = accuracy(preds, labels)

    return p.item(), r.item(), f.item(), a.item()


In [17]:
def evaluate_model(model, data_loader):
    """
    Evaluates the model on validation/test set and returns loss & metrics.
    """
    model.eval()
    total_loss = 0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for batch in data_loader:
            input_ids, attention_mask, token_type_ids, labels = (
                batch["input_ids"].to(device),
                batch["attention_mask"].to(device),
                batch["token_type_ids"].to(device),
                batch["labels"].to(device),
            )
            logits = model(input_ids, attention_mask, token_type_ids)
            loss = criterion(logits.view(-1, num_labels), labels.view(-1))
            total_loss += loss.item()

            # Collect predictions & labels
            all_preds.append(torch.argmax(logits, dim=-1).cpu())
            all_labels.append(labels.cpu())

    avg_loss = total_loss / len(data_loader)
    all_preds, all_labels = torch.cat(all_preds), torch.cat(all_labels)

    precision, recall, f1, accuracy = compute_metrics(all_preds, all_labels)
    return avg_loss, precision, recall, f1, accuracy

In [18]:
def train_model(model, train_loader, valid_loader, epochs=10, patience=3):
    """
    Trains the model with early stopping and prints training & validation metrics per epoch.
    """
    best_val_loss = float("inf")  # Track lowest validation loss
    patience_counter = 0  # Counter for early stopping

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        all_preds, all_labels = [], []

        for batch in train_loader:
            input_ids, attention_mask, token_type_ids, labels = (
                batch["input_ids"].to(device),
                batch["attention_mask"].to(device),
                batch["token_type_ids"].to(device),
                batch["labels"].to(device),
            )

            optimizer.zero_grad()
            logits = model(input_ids, attention_mask, token_type_ids)
            loss = criterion(logits.view(-1, num_labels), labels.view(-1))
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            # Collect predictions & labels for computing metrics
            all_preds.append(torch.argmax(logits, dim=-1).cpu())
            all_labels.append(labels.cpu())

        # Compute training loss & metrics
        avg_train_loss = total_loss / len(train_loader)
        all_preds, all_labels = torch.cat(all_preds), torch.cat(all_labels)
        train_precision, train_recall, train_f1, train_acc = compute_metrics(all_preds, all_labels)

        # Compute validation loss & metrics
        val_loss, val_precision, val_recall, val_f1, val_acc = evaluate_model(model, valid_loader)

        print(f"\nEpoch {epoch+1}/{epochs}:")
        print(f"Train Loss: {avg_train_loss:.4f} | Precision: {train_precision:.4f} | Recall: {train_recall:.4f} | F1: {train_f1:.4f} | Accuracy: {train_acc:.4f}")
        print(f"Valid Loss: {val_loss:.4f} | Precision: {val_precision:.4f} | Recall: {val_recall:.4f} | F1: {val_f1:.4f} | Accuracy: {val_acc:.4f}")

        # ✅ Early Stopping Check
        if val_loss < best_val_loss:
            best_val_loss = val_loss  # Update best loss
            patience_counter = 0  # Reset patience
            torch.save(model.state_dict(), "best_model2.pt")  # Save best model
            print("✅ Model saved!")
        else:
            patience_counter += 1  # Increment patience
            print(f"⏳ Patience Counter: {patience_counter}/{patience}")

        if patience_counter >= patience:  # Stop training if patience exceeded
            print("\n🛑 Early stopping triggered. Restoring best model...")
            model.load_state_dict(torch.load("best_model2.pt"))  # Load best model
            break

In [21]:
train_model(model, train_loader, valid_loader)


Epoch 1/10:
Train Loss: 0.0707 | Precision: 0.9595 | Recall: 0.9606 | F1: 0.9600 | Accuracy: 0.9782
Valid Loss: 0.0909 | Precision: 0.9424 | Recall: 0.9655 | F1: 0.9533 | Accuracy: 0.9739
✅ Model saved!

Epoch 2/10:
Train Loss: 0.0590 | Precision: 0.9685 | Recall: 0.9700 | F1: 0.9692 | Accuracy: 0.9831
Valid Loss: 0.0767 | Precision: 0.9586 | Recall: 0.9638 | F1: 0.9611 | Accuracy: 0.9788
✅ Model saved!

Epoch 3/10:
Train Loss: 0.0494 | Precision: 0.9736 | Recall: 0.9754 | F1: 0.9745 | Accuracy: 0.9860
Valid Loss: 0.0746 | Precision: 0.9601 | Recall: 0.9623 | F1: 0.9611 | Accuracy: 0.9789
✅ Model saved!

Epoch 4/10:
Train Loss: 0.0434 | Precision: 0.9771 | Recall: 0.9791 | F1: 0.9781 | Accuracy: 0.9879
Valid Loss: 0.0756 | Precision: 0.9613 | Recall: 0.9630 | F1: 0.9621 | Accuracy: 0.9796
⏳ Patience Counter: 1/3

Epoch 5/10:
Train Loss: 0.0365 | Precision: 0.9809 | Recall: 0.9825 | F1: 0.9817 | Accuracy: 0.9899
Valid Loss: 0.0930 | Precision: 0.9471 | Recall: 0.9695 | F1: 0.9576 | Acc

<ipython-input-18-44c7f01ea323>:57: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model2.pt"))  # Load best model


In [22]:
test_loss, test_precision, test_recall, test_f1, test_accuracy = evaluate_model(model, test_loader)

print("\nTest Evaluation:")
print(f"  Loss: {test_loss:.4f}")       # ✅ Now it won't crash                                                                                                                    
print(f"  Precision: {test_precision:.4f}")
print(f"  Recall: {test_recall:.4f}")
print(f"  F1-Score: {test_f1:.4f}")
print(f"  Accuracy: {test_accuracy:.4f}")


Test Evaluation:
  Loss: 0.0491
  Precision: 0.9724
  Recall: 0.9675
  F1-Score: 0.9698
  Accuracy: 0.9869


In [41]:
label_id_to_str = {
    0: "O",
    1: "X",
    2: "Y",
    # If you had more, add them here
}


In [42]:
def predict_on_text_charbert(model, text, tokenizer, char_tokenizer, max_seq_len=128):
    """
    Predict the NER tags for a single input text using CharBERT-style approach 
    (BERT subwords + grapheme-based character embeddings).
    
    Args:
        model: Trained CharBERT_NER model (with .bert + .char_encoder + .lstm).
        text: Raw Bengali text string.
        tokenizer: HuggingFace tokenizer (AutoTokenizer) for BERT subwords.
        char_tokenizer: Keras tokenizer trained on grapheme sequences.
        max_seq_len: Max sequence length used in training/fine-tuning.
    
    Returns:
        List of (word, predicted_label) tuples.
    """
    model.eval()

    # 1️⃣ Custom word-level splitting (punct, special chars)
    original_tokens = custom_tokenize(text)

    # 2️⃣ BERT subword tokenization + mapping
    bert_tokens = []
    token_mapping = []  # Maps subword indices back to original_tokens indices
    for idx, tok in enumerate(original_tokens):
        subwords = tokenizer.tokenize(tok)
        bert_tokens.extend(subwords)
        token_mapping.extend([idx] * len(subwords))

    # 3️⃣ Convert to BERT token IDs (CLS + subwords + SEP)
    input_ids = [tokenizer.cls_token_id] + tokenizer.convert_tokens_to_ids(bert_tokens) + [tokenizer.sep_token_id]
    token_mapping = [-1] + token_mapping + [-1]  # -1 for CLS and SEP

    # 4️⃣ Build Char/Grapheme input
    #    We only do one grapheme sequence per *original token*, then replicate for subwords.
    char_seq = []
    for word in original_tokens:
        # Split into graphemes
        g_list = grapheme_split(word)
        # Join with space so Keras char_tokenizer can parse them as separate tokens
        grapheme_text = " ".join(g_list)
        grapheme_ids = char_tokenizer.texts_to_sequences([grapheme_text])[0] if grapheme_text else []
        # Pad/truncate
        grapheme_ids = grapheme_ids[:MAX_CHAR_LEN]
        grapheme_ids += [0] * (MAX_CHAR_LEN - len(grapheme_ids))
        # We'll store, replicate later
        char_seq.append(grapheme_ids)

    #    Replicate char_seq for subwords + add CLS/SEP placeholders
    replicated_char_seq = [[0]*MAX_CHAR_LEN]  # CLS
    idx_char = 0
    for idx, tok in enumerate(original_tokens):
        subword_count = sum([1 for m in token_mapping if m == idx]) 
        # replicate the same grapheme vector subword_count times
        for _ in range(subword_count):
            replicated_char_seq.append(char_seq[idx_char])
        idx_char += 1
    replicated_char_seq.append([0]*MAX_CHAR_LEN)  # SEP

    # 5️⃣ Now we do sequence-level padding/truncation
    attention_mask = [1] * len(input_ids)

    if len(input_ids) > max_seq_len:
        input_ids = input_ids[:max_seq_len]
        attention_mask = attention_mask[:max_seq_len]
        token_mapping = token_mapping[:max_seq_len]
        replicated_char_seq = replicated_char_seq[:max_seq_len]
    else:
        pad_len = max_seq_len - len(input_ids)
        input_ids += [0]*pad_len
        attention_mask += [0]*pad_len
        token_mapping += [-1]*pad_len
        # Pad char_seq
        for _ in range(pad_len):
            replicated_char_seq.append([0]*MAX_CHAR_LEN)

    # 6️⃣ Convert to tensors
    input_ids      = torch.tensor([input_ids], dtype=torch.long).to(device)
    attention_mask = torch.tensor([attention_mask], dtype=torch.long).to(device)
    token_type_ids = torch.zeros_like(input_ids).to(device)  # typical for single-sentence inputs
    char_input     = torch.tensor([replicated_char_seq], dtype=torch.long).to(device)

    # 7️⃣ Forward pass with no grad
    with torch.no_grad():
        logits = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            char_input=char_input
        )
        preds = torch.argmax(logits, dim=-1).squeeze(0)  # shape: (seq_len,)

    # 8️⃣ Map subword predictions back to original tokens
    token_level_preds = {}
    for i, label_id in enumerate(preds.cpu().numpy()):
        orig_idx = token_mapping[i]
        if orig_idx == -1:
            continue  # skip CLS, SEP, padding
        # first subword of a token can represent the entire token
        # store only the first subword's predicted label
        if orig_idx not in token_level_preds:
            token_level_preds[orig_idx] = label_id

    # 9️⃣ Generate final output
    results = []
    for i, word in enumerate(original_tokens):
        label_id = token_level_preds.get(i, 0)  # default label if missing
        label_str = label_id_to_str[label_id]   # map ID → "O", "X", "Y", etc.
        results.append((word, label_str))

    return results


In [43]:
# Suppose you have a new text snippet
new_text = "রিনা ও মিঞা যথাক্রমে গণিতে ৫০ ও ৬৫ নম্বর পেয়েছে. কিন্তু শিমা পেয়েছে ৪০ নম্বর"
new_text="২০২০ সালে জিডিপি ছিল ৫.২%, কিন্তু মুদ্রাস্ফীতি ছিল ৩.১%। অন্যদিকে, ২০২১ সালে আমাদের জিডিপি বেড়ে ৬.৭% হলো, আর মুদ্রাস্ফীতি কমে দাঁড়ালো ২.১%।"
# Call predict
predictions = predict_on_text_charbert(model, new_text, tokenizer, char_tokenizer, max_seq_len=128)
tokens = [token for token, label in predictions]
labels = [label for token, label in predictions]

# Print token-label pairs
for token, label in predictions:
    print(f"{token}\t{label}")


২০২০	X
সালে	O
জিডিপি	X
ছিল	O
৫.২	Y
%	Y
,	O
কিন্তু	O
মুদ্রাস্ফীতি	X
ছিল	O
৩.১	Y
%	Y
।	O
অন্যদিকে	O
,	O
২০২১	X
সালে	O
আমাদের	O
জিডিপি	X
বেড়ে	O
৬.৭	Y
%	Y
হলো	O
,	O
আর	O
মুদ্রাস্ফীতি	X
কমে	O
দাঁড়ালো	O
২.১	Y
%	Y
।	O


In [ ]:
pip install llama-cpp-python

In [ ]:
import json
import subprocess
import os
from llama_cpp import Llama
from sentence_transformers import SentenceTransformer, util, CrossEncoder
import torch

In [ ]:
from llama_cpp import Llama

# ✅ Load the LLaMA 3 model from GGUF format
MODEL_PATH = "/kaggle/input/llma3_8b/pytorch/llama3_8b/1/Meta-Llama-3-8B-Instruct-Q6_K.gguf"
llm = Llama(model_path=MODEL_PATH, n_ctx=2048, n_gpu_layers=-1)  # Use GPU acceleration

def simplify_text(text):
    """Uses LLaMA-3 GGUF model to simplify numerical text intelligently."""
    
    prompt = f"""
    তুমি একটি কৃত্রিম বুদ্ধিমত্তা মডেল, যাকে বাংলা ভাষায় সংখ্যা সমৃদ্ধ একটি পাঠ্য দেওয়া হবে।
    তোমাকে ওই পাঠ্যের প্রাসঙ্গিক অর্থ বুঝতে হবে এবং তুলনামূলক বিবৃতিগুলোকে সুনির্দিষ্ট সংখ্যায় রূপান্তর করে পাঠ্যটি সরলীকরণ করতে হবে।
    **নিয়মাবলী:**
    ১. সংখ্যা পরিবর্তন কোরো না, শুধুমাত্র ব্যাখ্যা সহজ করো।
    ২. কোনো অতিরিক্ত তথ্য যোগ কোরো না।
    ৩. গাণিতিক হিসাব সঠিকভাবে সম্পন্ন করো।
    Input: রহিমের আছে ৪০টাকা, শফিকের তার দ্বিগুন এবং করিমের আছে রহিমের অর্ধেক। 
    Output: রহিমের কাছে আছে ৪০ টাকা, সফিকের কাছে আছে ৮০ টাকা, করিমের কাছে আছে ২০ টাকা।
    
    Input: প্রাণ বেভারেজ এই মাসে পূর্ববর্তী মাসের চেয়ে ২.৫% বেশি আয় করেছে, পূর্ববর্তী মাসে তাদের আয় ছিল ২,৫০,০০০ টাকা। পরবর্তী মাসে অনুমান করা হচ্ছে যে বর্তমান মাসের চেয়েও দ্বিগুণ বেশি আয় হবে।
    Output: প্রাণ বেভারেজ পূর্ববর্তী মাসে আয় করেছে ২,৫০,০০০ টাকা। এই মাসে আয় করেছে ২,৫৬,২৫০ টাকা। পরবর্তী মাসে অনুমান করা হচ্ছে যে আয় হবে ৫,০০,০০০ টাকা।
    
    Input: উত্তর আমেরিকায় এই বছর প্রচণ্ড গরম পড়েছে, যা গত দশকের তুলনায় ২.৫ ডিগ্রি সেলসিয়াস বেশি। গত দশকে তাপমাত্রা ছিল ৩৫ ডিগ্রি সেলসিয়াস।
    Output: উত্তর আমেরিকায় এই বছর প্রচণ্ড গরম পড়েছে। গত দশকে তাপমাত্রা ছিল ৩৫ ডিগ্রি সেলসিয়াস। এই বছর তাপমাত্রা হয়েছে ৩৭.৫ ডিগ্রি সেলসিয়াস।

    ---
    **ইনপুট:** "{text}"  
    **আউটপুট:**  
    """

    # ✅ Generate response using LLaMA-3 GGUF
    output = llm(
        prompt,
        max_tokens=512,  # More room for generated text
        temperature=0.1,  # Less randomness for accuracy
        top_p=0.9,  # Balanced output
        stop=['\n']  # Don't cut off early
    )

    # ✅ Extract and clean output
    simplified_text = output["choices"][0]["text"].strip().replace("\n", " ")

    # ✅ Ensure output is meaningful
    if not simplified_text or len(simplified_text.split()) < 3:
        return "Error: Model did not generate a valid response."

    return simplified_text

In [ ]:
def convert_to_test_sentences(text, max_words=150):
    """
    Splits a Bengali paragraph into chunks, each with at most `max_words`.
    Ensures words are not split in the middle.
    """
    words = text.split()  # Split the text into words
    chunks = []
    
    # ✅ Process in chunks of `max_words`
    for i in range(0, len(words), max_words):
        chunk_words = words[i:i+max_words]  # Extract a chunk of max_words
        chunk_text = " ".join(chunk_words)  # Convert back to text
        chunks.append(chunk_text.strip())
    print([chunk for chunk in chunks])
    return chunks

In [ ]:
def convert_to_test_sentences(text, max_words=150):
    """
    Splits a Bengali paragraph into chunks, each with at most `max_words`.
    Ensures words are not split in the middle.
    """
    words = text.split()  # Split the text into words
    chunks = []
    
    # ✅ Process in chunks of `max_words`
    for i in range(0, len(words), max_words):
        chunk_words = words[i:i+max_words]  # Extract a chunk of max_words
        chunk_text = " ".join(chunk_words)  # Convert back to text
        chunks.append(chunk_text.strip())
    print([chunk for chunk in chunks])
    return chunks

In [ ]:
def simplify_large_text(text):
    """
    Breaks large text into token-based chunks, processes each with LLaMA, and merges results.
    """
    chunks = convert_to_test_sentences(text)  # Token-based chunking
    simplified_chunks = [simplify_text(chunk) for chunk in chunks]  # Process each chunk
    
    return " ".join(simplified_chunks)  # Merge all processed chunks

In [ ]:
text="""সায়রা জুলাই মাসে ১০০০০ টাকা পায় তার ব্যবসা থেকে। আগস্ট মাসে তার লাভ হয় জুলাই মাস থেকে ৫০০০ টাকা বেশি। কিন্তু সেপ্টেম্বরে আবার জুলাই এর সমান পরিমাণ লাভ হয়। অক্টোবরে লাভের পরিমাণ সেপ্টেম্বর থেকে আরো ৩০০০ কমে ৭০০০ টাকা হয়ে যায়। নভেম্বরে আবার লাভের মুখ দেখে সায়রা। তার ব্যবসা থেকে বছরের শেষদিকে নভেম্বর মাসে লাভ হয় ২০০০০ টাকা।"""
# ✅ Process and get final simplified output
final_simplified_output = simplify_large_text(text)

# ✅ Print the final merged output
print("Final Simplified Output:", final_simplified_output)

In [ ]:
# Suppose you have a new text snippet
new_text = final_simplified_output

# Call predict
predictions = predict_on_text(model, new_text, tokenizer, max_seq_len=100)
tokens = [token for token, label in predictions]
labels = [label for token, label in predictions]

# Print token-label pairs
for token, label in predictions:
    print(f"{token}\t{label}")


In [ ]:
# Suppose you have a new text snippet
new_text = "২০২০ সালে জিডিপি ছিল ৫.২%, কিন্তু মুদ্রাস্ফীতি ছিল ৩.১%। অন্যদিকে, ২০২১ সালে আমাদের জিডিপি বেড়ে ৬.৭% হলো, আর মুদ্রাস্ফীতি কমে দাঁড়ালো ২.১%।"

# Call predict
predictions = predict_on_text(model, new_text, tokenizer, max_seq_len=100)
tokens = [token for token, label in predictions]
labels = [label for token, label in predictions]

# Print token-label pairs
for token, label in predictions:
    print(f"{token}\t{label}")
# Run validation
valid_pairs = validate_xy_pairs(tokens, labels)

# Output valid pairs
print("✅ Valid X-Y Pairs:", valid_pairs)

In [ ]:
# Suppose you have a new text snippet
new_text = "২০২০ সালে জিডিপি ছিল ৫.২%, কিন্তু মুদ্রাস্ফীতি ছিল ৩.১%। অন্যদিকে, ২০২১ সালে আমাদের জিডিপি বেড়ে ৬.৭% হলো, আর মুদ্রাস্ফীতি কমে গেলো।"

# Call predict
predictions = predict_on_text(model, new_text, tokenizer, max_seq_len=100)
tokens = [token for token, label in predictions]
labels = [label for token, label in predictions]

# Print token-label pairs
for token, label in predictions:
    print(f"{token}\t{label}")


In [ ]:
# Suppose you have a new text snippet
new_text = "কোভিড মহামারির বছর ২০২০ সালে ছয় হাজার ১৩০ জন আক্রান্ত হয় ম্যালেরিয়ায়। পরের বছর ২০২১ সালে এ সংখ্যা ছিল সাত হাজার ২৯৪।"

# Call predict
predictions = predict_on_text(model, new_text, tokenizer, max_seq_len=100)

# Print token-label pairs
for token, label in predictions:
    print(f"{token}\t{label}")


In [ ]:
# Suppose you have a new text snippet
new_text = "রিমা বার্ষিক পরীক্ষায় দ্বিতীয় হয়েছে কিন্তু সীমা হয়েছে তৃতীয়, ফজলি হয়েছে পঞ্চম"

# Call predict
predictions = predict_on_text(model, new_text, tokenizer, max_seq_len=100)

# Print token-label pairs
for token, label in predictions:
    print(f"{token}\t{label}")


In [ ]:
import pandas as pd
from typing import List
import re
txt="আমার কাছে তিন কোটি দুই লক্ষ পঞ্চাশ হাজার চারশো ত্রিশ টাকা আছে, তোমার কাছে আছে ৫০০০ টাকা, কিন্তু মিনার কাছে টাকা আছে ৫৪৯০৮"# Suppose you have a new text snippet
new_text = txt

# Call predict
predictions = predict_on_text(model, new_text, tokenizer, max_seq_len=100)
tokens = [token for token, label in predictions]
labels = [label for token, label in predictions]


# Print token-label pairs
for token, label in predictions:
    print(f"{token}\t{label}")

def convert_bengali_text_to_number(tokens, labels):
    """
    Converts Bengali number words to their numerical values based on the 'Y' labels.
    
    Args:
    - tokens (list): List of words (tokens) from the sentence.
    - labels (list): Corresponding NER labels (X, Y, O).

    Returns:
    - converted_tokens (list): Tokens with 'Y' values converted to numbers.
    """
    # Bengali number words mapping
    bengali_number_map = {
        "শূন্য": 0, "এক": 1, "দুই": 2, "তিন": 3, "চার": 4,
        "পাঁচ": 5, "ছয়": 6, "সাত": 7, "আট": 8, "নয়": 9,
        "দশ": 10, "এগারো": 11, "বারো": 12, "তেরো": 13, "চৌদ্দ": 14,
        "পনেরো": 15, "ষোল": 16, "সতেরো": 17, "আঠারো": 18, "ঊনিশ": 19,
        "বিশ": 20, "একুশ": 21, "বাইশ": 22, "তেইশ": 23, "চব্বিশ": 24,
        "পঁচিশ": 25, "ছাব্বিশ": 26, "সাতাশ": 27, "আঠাশ": 28, "ঊনত্রিশ": 29,
        "ত্রিশ": 30, "একত্রিশ": 31, "বত্রিশ": 32, "তেত্রিশ": 33, "চৌত্রিশ": 34,
        "পঁইত্রিশ": 35, "ছত্রিশ": 36, "সাইত্রিশ": 37, "আটত্রিশ": 38, "ঊনচল্লিশ": 39,
        "চল্লিশ": 40, "একচল্লিশ": 41, "বিয়াল্লিশ": 42, "তেতাল্লিশ": 43, "চুয়াল্লিশ": 44,
        "পঁইচল্লিশ": 45, "ছিচল্লিশ": 46, "সাতচল্লিশ": 47, "আটচল্লিশ": 48, "ঊনপঞ্চাশ": 49,
        "পঞ্চাশ": 50, "একান্ন": 51, "বায়ান্ন": 52, "তিপ্পান্ন": 53, "চুয়ান্ন": 54,
        "পঞ্চান্ন": 55, "ছাপ্পান্ন": 56, "সাতান্ন": 57, "আটান্ন": 58, "ঊনষাট": 59,
        "ষাট": 60, "একষট্টি": 61, "বাষট্টি": 62, "তেষট্টি": 63, "চৌষট্টি": 64,
        "পঁইষট্টি": 65, "ছেষট্টি": 66, "সাতষট্টি": 67, "আটষট্টি": 68, "ঊনসত্তর": 69,
        "সত্তর": 70, "একাত্তর": 71, "বাহাত্তর": 72, "তিয়াত্তর": 73, "চুয়াত্তর": 74,
        "পঁচাত্তর": 75, "ছিয়াত্তর": 76, "সাতাত্তর": 77, "আটাত্তর": 78, "ঊনআশি": 79,
        "আশি": 80, "একাশি": 81, "বিরাশি": 82, "তিরাশি": 83, "চুরাশি": 84,
        "পঁচাশি": 85, "ছিয়াশি": 86, "সাতাশি": 87, "আটাশি": 88, "ঊননব্বই": 89,
        "নব্বই": 90, "একানব্বই": 91, "বিরানব্বই": 92, "তিরানব্বই": 93, "চুরানব্বই": 94,
        "পঁচানব্বই": 95, "ছিয়ানব্বই": 96, "সাতানব্বই": 97, "আটানব্বই": 98, "নিরানব্বই": 99,
        "একশো": 100, "দুইশো": 200, "তিনশো": 300, "চারশো": 400, "পাঁচশো": 500,
        "ছয়শো": 600, "সাতশো": 700, "আটশো": 800, "নয়শো": 900
    }

    multipliers = {
        "হাজার": 1000, "লক্ষ": 100000, "লাখ": 100000, "কোটি": 10000000,
        "মিলিয়ন": 1000000, "বিলিয়ন": 1000000000
    }

    # Convert words to numbers
    converted_tokens = []
    current_number = 0
    i = 0

    while i < len(tokens):
        word = tokens[i]
        label = labels[i]

        if label == "Y":
            if word.isdigit():  # If it's already a number, store as integer
                current_number += int(word)
            elif word in bengali_number_map:  # Convert Bengali number words
                temp_number = bengali_number_map[word]

                # Look ahead for a multiplier
                if (i + 1) < len(tokens) and tokens[i + 1] in multipliers:
                    temp_number *= multipliers[tokens[i + 1]]
                    i += 1  # Skip the multiplier

                current_number += temp_number
            else:
                converted_tokens.append(word)  # Keep other words unchanged
        else:
            if current_number > 0:
                converted_tokens.append(current_number)
                current_number = 0  # Reset for next number
            converted_tokens.append(word)

        i += 1

    # If there's an unprocessed number at the end
    if current_number > 0:
        converted_tokens.append(current_number)

    return converted_tokens


# Run the function
token,labels=valid_pairs = validate_xy_pairs(tokens, labels)

# Output valid pairs
print("✅ Valid X-Y Pairs:", valid_pairs)
converted_tokens = convert_bengali_text_to_number(tokens, labels)


print("✅ Converted Tokens:", converted_tokens)


In [ ]:
import re
import pandas as pd
from typing import List

def convert_bengali_text_to_number(tokens, labels):
    """Pairs `X` labels with `Y` words and prepares them for conversion."""
    
    # Define Bengali Number Words and Multipliers
    bengali_number_map = {
        "শূন্য": 0, "এক": 1, "দুই": 2, "তিন": 3, "চার": 4, "পাঁচ": 5, "ছয়": 6, "সাত": 7, "আট": 8, "নয়": 9,
        "দশ": 10, "বিশ": 20, "ত্রিশ": 30, "চল্লিশ": 40, "পঞ্চাশ": 50, "ষাট": 60, "সত্তর": 70, "আশি": 80, "নব্বই": 90,
        "একশো": 100, "দুইশো": 200, "তিনশো": 300, "চারশো": 400, "পাঁচশো": 500, "ছয়শো": 600, "সাতশো": 700, "আটশো": 800, "নয়শো": 900
    }

    multipliers = {
        "হাজার": 1000, "লক্ষ": 100000, "লাখ": 100000, "কোটি": 10000000, "মিলিয়ন": 1000000, "বিলিয়ন": 1000000000
    }

    paired_results = {}  # Store results
    current_x = None
    current_y_words = []  # Store all Y words
    unit = None  # Store unit if found
    last_multiplier = None  # Track last multiplier to prevent duplication
    i = 0

    while i < len(tokens):
        word, label = tokens[i], labels[i]

        if label == "X":
            if current_x and current_y_words:  
                # Store previous X-Y pair with its unit (before conversion)
                paired_results[current_x] = {"words": current_y_words, "unit": unit}
                current_y_words = []  
                unit = None  
                last_multiplier = None  # Reset last multiplier

            current_x = word  # Assign new X entity

        elif label == "Y":
            current_y_words.append(word)  # Store Y word

            # Check if it's a multiplier and store only once
            if word in multipliers and last_multiplier != word:
                unit = word  # Store the first occurrence as the unit
                last_multiplier = word  # Prevent duplicate multipliers

        i += 1  

    # Store the last X-Y pair
    if current_x and current_y_words:
        paired_results[current_x] = {"words": current_y_words, "unit": unit}

    return paired_results, bengali_number_map, multipliers


def convert_text_to_number(paired_results, bengali_number_map, multipliers):
    """
    Converts Bengali number words into numeric values while keeping the unit.

    Args:
    - paired_results (dict): Mapping of `X` to `Y` words before conversion.
    - bengali_number_map (dict): Mapping of Bengali words to numbers.
    - multipliers (dict): Mapping of Bengali multipliers (হাজার, কোটি, etc.).

    Returns:
    - converted_results (dict): Mapping of `X` to final numeric values with units.
    """
    converted_results = {}

    for x_entity, data in paired_results.items():
        y_words = data["words"]
        unit = data["unit"] if data["unit"] else ""

        # Convert Bengali number words to a single numerical value
        number_text = " ".join(y_words)
        number_value = convert_bengali_text_to_number_helper(number_text, bengali_number_map, multipliers)

        # Store the final result
        converted_results[x_entity] = f"{number_value} {unit}"

    return converted_results


def convert_bengali_text_to_number_helper(number_text, bengali_number_map, multipliers):
    """
    Helper function to convert Bengali text numbers into numeric values.

    Args:
    - number_text (str): Bengali number words.
    - bengali_number_map (dict): Mapping of Bengali words to numbers.
    - multipliers (dict): Mapping of Bengali multipliers (হাজার, কোটি, etc.).

    Returns:
    - int: Converted numeric value.
    """
    words = number_text.split()
    total = 0
    temp_value = 0

    for i, word in enumerate(words):
        if word.isdigit():
            temp_value += int(word)
        elif word in bengali_number_map:
            temp_value += bengali_number_map[word]
        elif word in multipliers:
            temp_value *= multipliers[word]
            total += temp_value
            temp_value = 0

    return total + temp_value


# Example Input
tokens = ["আমার", "কাছে", "তিন", "কোটি", "দুই", "লক্ষ", "পঞ্চাশ", "হাজার", "চারশো", "ত্রিশ", "টাকা", "আছে",
          ",", "তোমার", "কাছে", "আছে", "৫০০০", "টাকা", ",", "কিন্তু", "মিনার", "কাছে", "টাকা", "আছে", "৫৪৯০৮"]

labels = ["X", "O", "Y", "Y", "Y", "Y", "Y", "Y", "Y", "Y", "Y", "O",
          "O", "X", "O", "O", "Y", "Y", "O", "O", "X", "O", "O", "O", "Y"]

# Step 1: Pair `X` with `Y` words before conversion
paired_results, bengali_number_map, multipliers = convert_bengali_text_to_number(tokens, labels)

# Step 2: Convert `Y` words into numeric values
final_results = convert_text_to_number(paired_results, bengali_number_map, multipliers)

# Output the result
print("🔹 **Before Conversion:**")
for key, data in paired_results.items():
    print(f"{key} -> {' '.join(data['words'])} {data['unit'] if data['unit'] else ''}")

print("\n🔹 **After Conversion:**")
for key, value in final_results.items():
    print(f"{key} -> {value}")


In [ ]:
import pandas as pd
from typing import List
import re

# Example text
txt = "আমার কাছে তিন কোটি দুই লক্ষ পঞ্চাশ হাজার চারশো ত্রিশ টাকা আছে, তোমার কাছে আছে ৫০০০ টাকা, কিন্তু মিনার কাছে টাকা আছে ৫৪৯০৮"

# Call predict function (ensure predict_on_text returns [(token, label), ...])
predictions = predict_on_text(model, txt, tokenizer, max_seq_len=100)

# Extract tokens and labels
tokens = [token for token, label in predictions]
labels = [label for token, label in predictions]

def extract_xy_pairs(data):
    pairs = {}
    current_x = None
    current_y = []
    
    for word, label in data:
        if label == 'X':
            if current_x and current_y:
                pairs[current_x] = ' '.join(current_y)
            current_x = word
            current_y = []
        elif label == 'Y':
            if current_x:
                current_y.append(word)
    
    if current_x and current_y:
        pairs[current_x] = ' '.join(current_y)
    
    return pairs

# Extract pairs
xy_pairs = extract_xy_pairs(predictions)  # Directly pass predictions

# Print the result
for x, y in xy_pairs.items():
    print(f"{x} -> {y}")


In [ ]:
import re
import pandas as pd
from typing import List
import plotly.graph_objects as go


def extract_xy_pairs(data):
    pairs = {}
    units = {}
    current_x = None
    current_y = []
    current_unit = None
    unit_list = []
    
    # Define Bengali Number Words and Multipliers
    bengali_number_map = {
        "শূন্য": 0, "এক": 1, "দুই": 2, "তিন": 3, "চার": 4, "পাঁচ": 5, "ছয়": 6, "সাত": 7, "আট": 8, "নয়": 9,
        "দশ": 10, "বিশ": 20, "ত্রিশ": 30, "চল্লিশ": 40, "পঞ্চাশ": 50, "ষাট": 60, "সত্তর": 70, "আশি": 80, "নব্বই": 90,
        "একশো": 100, "দুইশো": 200, "তিনশো": 300, "চারশো": 400, "পাঁচশো": 500, "ছয়শো": 600, "সাতশো": 700, "আটশো": 800, "নয়শো": 900
    }

    multipliers = {
        "হাজার": 1000, "লক্ষ": 100000, "লাখ": 100000, "কোটি": 10000000, "মিলিয়ন": 1000000, "বিলিয়ন": 1000000000
    }
    
    for i, (word, label) in enumerate(data):
        if label == 'X':
            if current_x and current_y:
                pairs[current_x] = ' '.join(current_y)
                units[current_x] = current_unit
                unit_list.append(current_unit if current_unit else "null")
            current_x = word
            current_y = []
            current_unit = None
        elif label == 'Y':
            if current_x:
                current_y.append(word)
            if word not in bengali_number_map and word not in multipliers:
                if not word.isdigit():  # Ensure it's not a number before marking it as a unit
                    current_unit = word
    
    if current_x and current_y:
        pairs[current_x] = ' '.join(current_y)
        units[current_x] = current_unit
        unit_list.append(current_unit if current_unit else "null")
    
    return pairs, units, unit_list

def normalize_units(numeric_results, unit_list):
    """Converts all values to the smallest detected unit if necessary."""
    conversion_map = {"বিলিয়ন": 1000, "মিলিয়ন": 1, "লক্ষ": 0.1}  # Convert billions to millions
    
    # Detect the most frequently occurring unit (smallest denominator)
    valid_units = [unit for unit in unit_list if unit in conversion_map]
    if not valid_units:
        return numeric_results, unit_list  # No unit conversions needed
    
    target_unit = min(valid_units, key=lambda x: conversion_map[x])  # Find lowest unit
    normalized_results = {}
    new_unit_list = []
    
    for (key, value), unit in zip(numeric_results.items(), unit_list):
        if unit in conversion_map:
            factor = conversion_map[unit] / conversion_map[target_unit]
            normalized_results[key] = value * factor
            new_unit_list.append(target_unit)
        else:
            normalized_results[key] = value
            new_unit_list.append(unit)
    
    return normalized_results, new_unit_list
def parse_bengali_float(token: str):
    """
    Translates Bengali digits in the token to ASCII digits,
    then attempts to convert it to a float.
    Returns None if unsuccessful.
    """
    # Make a translation map for Bengali digits -> ASCII digits
    translation_map = str.maketrans("০১২৩৪৫৬৭৮৯", "0123456789")
    
    # Translate the token to ASCII form
    ascii_token = token.translate(translation_map)
    
    try:
        return float(ascii_token)  # If "৫.২" => "5.2" => 5.2 (float)
    except ValueError:
        return None


def convert_bengali_text_to_number(xy_pairs_dict, units):
    """Converts Bengali number words into numerical values and retains units."""
    bengali_number_map = {
        'এক': 1, 'প্রথম': 1, 'পহেলা': 1,
        'দুই': 2, 'দ্বিতীয়': 2, 'দোসরা': 2,
        'তিন': 3, 'তৃতীয়': 3,
        'চার': 4, 'চতুর্থ': 4,
        'পাঁচ': 5, 'পঞ্চম': 5,
        'ছয়': 6, 'ষষ্ঠ': 6,
        'সাত': 7, 'সপ্তম': 7,
        'আট': 8, 'অষ্টম': 8,
        'নয়': 9, 'নবম': 9,
        'দশ': 10, 'দশম': 10, "এগারো": 11, "বারো": 12, "তেরো": 13, "চৌদ্দ": 14,
        "পনেরো": 15, "ষোল": 16, "সতেরো": 17, "আঠারো": 18, "ঊনিশ": 19,
        "বিশ": 20, "একুশ": 21, "বাইশ": 22, "তেইশ": 23, "চব্বিশ": 24,
        "পঁচিশ": 25, "ছাব্বিশ": 26, "সাতাশ": 27, "আঠাশ": 28, "ঊনত্রিশ": 29,
        "ত্রিশ": 30, "একত্রিশ": 31, "বত্রিশ": 32, "তেত্রিশ": 33, "চৌত্রিশ": 34,
        "পঁইত্রিশ": 35, "ছত্রিশ": 36, "সাইত্রিশ": 37, "আটত্রিশ": 38, "ঊনচল্লিশ": 39,
        "চল্লিশ": 40, "একচল্লিশ": 41, "বিয়াল্লিশ": 42, "তেতাল্লিশ": 43, "চুয়াল্লিশ": 44,
        "পঁইচল্লিশ": 45, "ছিচল্লিশ": 46, "সাতচল্লিশ": 47, "আটচল্লিশ": 48, "ঊনপঞ্চাশ": 49,
        "পঞ্চাশ": 50, "একান্ন": 51, "বায়ান্ন": 52, "তিপ্পান্ন": 53, "চুয়ান্ন": 54,
        "পঞ্চান্ন": 55, "ছাপ্পান্ন": 56, "সাতান্ন": 57, "আটান্ন": 58, "ঊনষাট": 59,
        "ষাট": 60, "একষট্টি": 61, "বাষট্টি": 62, "তেষট্টি": 63, "চৌষট্টি": 64,
        "পঁইষট্টি": 65, "ছেষট্টি": 66, "সাতষট্টি": 67, "আটষট্টি": 68, "ঊনসত্তর": 69,
        "সত্তর": 70, "একাত্তর": 71, "বাহাত্তর": 72, "তিয়াত্তর": 73, "চুয়াত্তর": 74,
        "পঁচাত্তর": 75, "ছিয়াত্তর": 76, "সাতাত্তর": 77, "আটাত্তর": 78, "ঊনআশি": 79,
        "আশি": 80, "একাশি": 81, "বিরাশি": 82, "তিরাশি": 83, "চুরাশি": 84,
        "পঁচাশি": 85, "ছিয়াশি": 86, "সাতাশি": 87, "আটাশি": 88, "ঊননব্বই": 89,
        "নব্বই": 90, "একানব্বই": 91, "বিরানব্বই": 92, "তিরানব্বই": 93, "চুরানব্বই": 94,
        "পঁচানব্বই": 95, "ছিয়ানব্বই": 96, "সাতানব্বই": 97, "আটানব্বই": 98, "নিরানব্বই": 99,
        "একশো": 100, "দুইশো": 200, "তিনশো": 300, "চারশো": 400, "পাঁচশো": 500,
        "ছয়শো": 600, "সাতশো": 700, "আটশো": 800, "নয়শো": 900
    }

    multipliers = {
        "হাজার": 1000, "লক্ষ": 100000, "লাখ": 100000, "কোটি": 10000000,
        "মিলিয়ন": 1000000, "বিলিয়ন": 1000000000
    }
    
    numeric_dict = {}
    for x_label, y_text in xy_pairs_dict.items():
        tokens = y_text.split()
        total_value = 0
        part_sum = 0

        for w in tokens:
            # 1) Try integer
            if w.isdigit():
                part_sum += int(w)

            # 2) Try dictionary
            elif w in bengali_number_map:
                part_sum += bengali_number_map[w]

            # 3) Try multiplier
            elif w in multipliers:
                part_sum *= multipliers[w]
                total_value += part_sum
                part_sum = 0

            # 4) Attempt to parse decimal (float)
            else:
                val = parse_bengali_float(w)
                if val is not None:
                    part_sum += val
                else:
                    # Possibly a unit or unknown word
                    pass
        
        # Once we finish all tokens for this X, update total_value and the dictionary
        total_value += part_sum
        numeric_dict[x_label] = total_value

    # Now we have processed **all** X-Y pairs
    return numeric_dict

    
def get_dynamic_ticks(min_val: float, max_val: float):
    """
    Generates y-axis tick values divided into 10 equal steps based on min and max values.
    """
    if max_val <= min_val:
        return [min_val]  # Edge case

    step = (max_val - min_val) // 10
    ticks = [round(min_val + i * step, 2) for i in range(11)]  # 11 ticks = 10 steps
    return ticks

def convert_to_bengali(num):
    bengali_digits = '০১২৩৪৫৬৭৮৯'
    return ''.join(bengali_digits[int(digit)] if digit.isdigit() else digit for digit in str(num))


# # Extract pairs and units
# xy_pairs, xy_units, unit_list = extract_xy_pairs(predictions)

# # Convert Y values to numeric format
# numeric_values = convert_bengali_text_to_number(xy_pairs, xy_units)

# # Normalize units
# normalized_values, normalized_units = normalize_units(numeric_values, unit_list)

# # Print results
# for idx, (x, y) in enumerate(normalized_values.items(), 1):
#     print(f"{x} -> {y}")

# # Print unit mapping
# for idx, unit in enumerate(normalized_units, 1):
#     print(f"unit{idx} -> {unit}")


In [ ]:
# Convert Y values to numeric format
# numeric_values = convert_bengali_text_to_number(xy_pairs, xy_units)

# Convert values to Bengali numerals for display
x_values = ['গত দশকে', 'আগামী পাঁচ বছরে']
y_values = [75,200]
unit_list = ['মিলিয়ন', 'মিলিয়ন']

y_values_bengali = [convert_to_bengali(value) + " " + unit for value, unit in zip(y_values, unit_list)]

max_val = max(y_values) if y_values else 0
min_val = min(y_values) if y_values else 0
print(min_val)
print(max_val)

y_tick_vals = get_dynamic_ticks(min_val, max_val)
y_tick_text_bengali = [convert_to_bengali(val) for val in y_tick_vals]

# Create the Plotly bar chart
fig = go.Figure([go.Bar(
    x=x_values,
    y=y_values,
    width=0.6,
    text=y_values_bengali,
    textfont=dict(size=15),
    textposition='auto',
    marker=dict(color='blue')
)])

# Update layout with Bengali tick labels and fonts
fig.update_layout(
    xaxis=dict(
        tickfont=dict(size=13),
        title=dict(text="নাম", font=dict(size=13))  # X-axis label in Bengali
    ),
    yaxis=dict(
        tickvals=y_tick_vals,
        ticktext=y_tick_text_bengali,
        tickfont=dict(size=15),
        title=dict(text="মান", font=dict(size=13))  # Y-axis label in Bengali
    ),
    width=600,
    height=400,
    margin=dict(l=50, r=50, t=50, b=50)
)

# Show and save the plot
fig.show()
fig.write_html('generated_chart.html')

In [ ]:
(Moyna has 53 taka. Rohima has 159 taka. Mohona has 318 taka. Hena has 79.5 taka.)
